In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import importlib

import torch
import torch.nn as nn
from torch_geometric.data import DataLoader

import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath('../../code/'))
import utils
from utils.functional_graphs import create_graph_dataset
from utils.models import Attention3Conv
from utils.traning_functions import train_process, test_process

In [3]:
importlib.reload(utils)

<module 'utils' (namespace)>

In [4]:
import warrior as war

WARRIOR
   brain initialized
   network initialized


In [5]:
fcs = np.load('../../local/fcs/combined/sparse_fcs_1316_subjects.npy')
print(fcs.shape)
labels = np.load('../../local/gender_labels/combined_gender_labels_1316_subjects.npy')
print(labels.shape)

(1316, 100, 100)
(1316,)


In [6]:
N_GRAPHS = fcs.shape[0]
PERCENT = 0.75 
N_TRAINING_SAMPLES = int(np.floor(PERCENT*N_GRAPHS))
#N_TRAINING_SAMPLES, N_TRAINING_SAMPLES/N_GRAPHS

In [7]:
root = '../../local/graph_datasets/hcp_1316/'
dataset = create_graph_dataset(sparse_fcs=fcs, root=root, labels=labels)

In [8]:
dataset_shuffled = dataset.shuffle()
train_dataset = dataset_shuffled[:N_TRAINING_SAMPLES]
test_dataset = dataset_shuffled[N_TRAINING_SAMPLES:]
print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 987
Number of test graphs: 329


In [9]:
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)#, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)#, shuffle=False)

In [10]:
my_dimensions = [64, 128, 256]
my_learning_rates = [.0005, .0001]
EPOCHS = 101

In [11]:
for p1 in my_dimensions:
    for p2 in my_learning_rates:
        print(f'Paremeters: {p1, p2}')
        model = Attention3Conv(num_node_features=dataset.num_features, hidden_channels=p1, dropout_rate=0.5, edge_dropout_rate=0.1)
        metrics = np.empty((2,EPOCHS))
        optimizer = torch.optim.AdamW(model.parameters(),lr=p2,  weight_decay=1e-4, amsgrad=True)
        criterion = nn.BCEWithLogitsLoss()
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to(device)
        for epoch in range(EPOCHS):
            train_process(model, train_loader, optimizer, criterion, device)
            train_acc = test_process(model, train_loader, device)
            test_acc = test_process(model, test_loader, device)
            metrics[0,epoch] = train_acc
            metrics[1,epoch] = test_acc
            #if epoch % 10 == 0:
                #print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
                
        np.save(f'results/fc_data/metrics/3conv/fc3c_metrics_dim{p1}_lr{p2}.npy', metrics)
        torch.save(model.state_dict(), f'results/fc_data/models/3conv/fc3c_model_dim{p1}_lr{p2}')

Paremeters: (0.3, 0.3)


KeyboardInterrupt: 